In [25]:
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score

# ============================
# 1) CARGAR DATASET
# ============================
df = pd.read_csv("health_data_processed.csv")
df.head()

,Edad,Género,Estado civil,Altura,Peso,Índice de masa corporal,¿Fuma actualmente?,¿Fumó en el pasado?,¿Consume alcohol frecuentemente?,Nivel de actividad física,...,Asma_real,Cáncer_real,Obesidad_real,Depresión/Ansiedad_real,Enfermedad cardiovascular_bin,Diabetes_bin,Asma_bin,Cáncer_bin,Obesidad_bin,Depresión/Ansiedad_bin
0,76.596326,Otro,Soltero,153.681426,76.920289,29.612895,No,Sí,No,Moderado,...,0.607766,0.206125,0.185332,0.526435,1,0,1,0,0,1
1,79.795297,Otro,Casado,155.882307,66.743641,9.902543,No,No,No,Moderado,...,0.127169,0.464862,0.353357,1.062676,0,0,0,0,0,1
2,90.603394,Otro,Casado,176.481841,124.818134,27.248719,Sí,No,Sí,Moderado,...,0.518238,0.152951,0.233972,0.290081,1,0,1,0,0,0
3,22.154276,Femenino,Viudo,158.681358,114.807668,27.634473,No,No,No,Moderado,...,0.087128,0.016280,0.908137,0.561079,1,0,0,0,1,1
4,46.176676,Masculino,Casado,184.451263,60.217207,24.094841,No,Sí,No,Sedentario,...,0.216952,0.110858,0.713157,1.180575,1,0,0,0,1,1


In [26]:
# ============================
# 2) DEFINIR TARGETS
# ============================
target_cols = [
    "Asma_bin", "Cáncer_bin", "Obesidad_bin",
    "Depresión/Ansiedad_bin", "Diabetes_bin",
    "Enfermedad cardiovascular_bin"
]

X = df.drop(columns=target_cols)
y = df[target_cols]

# One-hot encoding
X = pd.get_dummies(X, drop_first=True)

# Asegurar 2D
X = X.values
y = y.values

if X.ndim == 3:
    X = X.reshape(X.shape[0], -1)

print(X.shape, y.shape)

(20000, 74) (20000, 6)


In [27]:
# ============================
# 3) TRAIN/TEST SPLIT + ESCALADO
# ============================
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5)


print('Train:', X_train.shape, y_train.shape)
print('Test:', X_test.shape, y_test.shape)
print('Valid:', X_valid.shape, y_valid.shape)

Train: (14000, 74) (14000, 6)
Test: (3000, 74) (3000, 6)
Valid: (3000, 74) (3000, 6)


In [ ]:
# ============================
# 4) DEFINIR ALGORTIMO GENÉTICO
# ============================
num_features = X_train.shape[1]

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# gen binario
toolbox.register("attr_bool", lambda: random.randint(0, 1))
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, n=num_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluar(ind):
    mask = np.array(ind, dtype=bool)
    if mask.sum() == 0:
        return 0.0,

    Xtr = X_train[:, mask]
    Xts = X_valid[:, mask]

    clf = MultiOutputClassifier(RandomForestClassifier(n_estimators=150))
    clf.fit(Xtr, y_train)
    pred = clf.predict(Xts)

    score = f1_score(y_valid, pred, average="macro")
    
    return score,

toolbox.register("evaluate", evaluar)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.02)
toolbox.register("select", tools.selTournament, tournsize=3)

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Se implementa la configuración completa del algoritmo genético encargado de seleccionar las mejores características para el modelo de clasificación multietiqueta. Primero se define el tipo de individuo como una lista de valores binarios, donde cada bit indica si una característica es incluida (1) o descartada (0). Luego se registra la función de evaluación, la cual recibe un individuo, aplica su máscara sobre los datos de entrenamiento y validación, entrena un clasificador Random Forest dentro de un MultiOutputClassifier, y finalmente devuelve el puntaje F1 macro obtenido en el conjunto de validación. De esta manera, cada individuo representa una combinación posible de características, y su fitness corresponde al rendimiento del modelo usando únicamente ese subconjunto. Además, se establecen los operadores genéticos fundamentales: cruce de dos puntos (cxTwoPoint), mutación por volteo de bits (mutFlipBit) y selección mediante torneo (selTournament), lo que permite que la población evolucione y converja hacia combinaciones de características cada vez más efectivas.

In [29]:
# ============================
# 5) CORRER GA
# ============================
population = toolbox.population(n=40)
NGEN = 15

result = algorithms.eaSimple(population, toolbox,
                              cxpb=0.5, mutpb=0.2,
                              ngen=NGEN, verbose=True)

best = tools.selBest(population, 1)[0]
best

gen	nevals
0  	40    
1  	25    
2  	16    
3  	19    
4  	23    
5  	24    
6  	23    
7  	23    
8  	31    
9  	19    
10 	24    
11 	20    
12 	31    
13 	24    
14 	21    
15 	25    


[0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0]

In [30]:
# ============================
# 6) ENTRENAR MODELO FINAL CON MEJORES FEATURES
# ============================
best_mask = np.array(best, dtype=bool)

clf_final = MultiOutputClassifier(RandomForestClassifier(n_estimators=200))
clf_final.fit(X_train[:, best_mask], y_train)

pred_final = clf_final.predict(X_test[:, best_mask])

f1_final = f1_score(y_test, pred_final, average="macro")
print('F1 final:', f1_final)

F1 final: 0.9998876151944257


In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

# =============================
# PREPARAR NOMBRES CORRECTOS
# =============================

target_names = target_cols  # nombres reales de las enfermedades
feature_names = pd.get_dummies(df.drop(columns=target_cols), drop_first=True).columns


# =============================
# METRICAS COMPLETAS
# =============================
print("=== CLASSIFICATION REPORT ===\n")
print(classification_report(y_test, pred_final, target_names=target_names))

print("\n=== CONFUSION MATRICES ===")
cm = multilabel_confusion_matrix(y_test, pred_final)
for i, etiqueta in enumerate(target_names):
    print(f"\nMatriz para {etiqueta}:")
    print(cm[i])



=== CLASSIFICATION REPORT ===

                               precision    recall  f1-score   support

                     Asma_bin       1.00      1.00      1.00       352
                   Cáncer_bin       1.00      1.00      1.00        45
                 Obesidad_bin       1.00      1.00      1.00       741
       Depresión/Ansiedad_bin       1.00      1.00      1.00      1651
                 Diabetes_bin       1.00      1.00      1.00       562
Enfermedad cardiovascular_bin       1.00      1.00      1.00      1042

                    micro avg       1.00      1.00      1.00      4393
                    macro avg       1.00      1.00      1.00      4393
                 weighted avg       1.00      1.00      1.00      4393
                  samples avg       0.84      0.84      0.84      4393


=== CONFUSION MATRICES ===

Matriz para Asma_bin:
[[2648    0]
 [   0  352]]

Matriz para Cáncer_bin:
[[2955    0]
 [   0   45]]

Matriz para Obesidad_bin:
[[2258    1]
 [   0  741]]



c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [ ]:
# =============================
# PREDICCIÓN ALEATORIA
# =============================
idx = np.random.randint(0, len(X_test))
sample_X = X_test[idx]
sample_y = y_test[idx]

print("\n\n==============================")
print("   PREDICCIÓN ALEATORIA")
print("==============================\n")

print("Index seleccionado:", idx)

print("\n--- FEATURES UTILIZADOS (solo las seleccionadas por el GA) ---")
print(pd.Series(sample_X[best_mask], index=feature_names[best_mask]))

pred_sample = clf_final.predict(sample_X[best_mask].reshape(1, -1))[0]

print("\n--- VERDAD REAL ---")
print(pd.Series(sample_y, index=target_names))

print("\n--- PREDICCIÓN DEL MODELO ---")
print(pd.Series(pred_sample, index=target_names))



   PREDICCIÓN ALEATORIA

Index seleccionado: 672

--- FEATURES UTILIZADOS (solo las seleccionadas por el GA) ---
Altura                                                           -1.417775
Peso                                                             -2.215183
Índice de masa corporal                                           0.001121
Enfermedad cardiovascular                                         1.227417
Diabetes                                                         -0.863870
Asma                                                             -0.196777
Obesidad                                                          0.123017
Depresión/Ansiedad                                                2.110329
Diabetes_real                                                    -0.863870
Cáncer_real                                                      -0.405813
Obesidad_real                                                     0.123017
Depresión/Ansiedad_real                                     

# Concluciones

Los resultados obtenidos muestran un rendimiento excepcionalmente alto del modelo entrenado con selección de características mediante algoritmo genético y un clasificador Random Forest en configuración multietiqueta. El classification report revela valores perfectos de precisión, recall y F1-score (1.00) en las seis enfermedades evaluadas, lo cual indica que el modelo predice correctamente tanto los casos positivos como negativos sin cometer errores en este conjunto de prueba. Las matrices de confusión confirman esta situación: para cada etiqueta, todas las instancias fueron clasificadas en la categoría correcta, con cero falsos positivos y cero falsos negativos, salvo un caso aislado en la etiqueta de obesidad. Este tipo de desempeño, aunque deseable, también puede ser indicativo de un posible sobreajuste si el conjunto de validación no es suficientemente diverso o si existe fuga de información durante el preprocesamiento. No obstante, desde una perspectiva puramente métrica, el modelo demuestra una capacidad sobresaliente para discriminar cada una de las condiciones de salud analizadas.